In [1]:
import os
import cv2
import random
import numpy as np

In [2]:
def augment_input(img):
    random_limits = (0.9, 1.1)
    y, x = img.shape[:2]
    fx = float(x)
    fy = float(y)
    src_point = np.float32([[fx / 2, fy / 3, ],
                            [2 * fx / 3, 2 * fy / 3],
                            [fx / 3, 2 * fy / 3]])
    random_shift = (np.random.rand(3, 2) - 0.5) * 2 * (random_limits[1] - random_limits[0]) / 2 + np.mean(random_limits)
    dst_point = src_point * random_shift.astype(np.float32)
    transform = cv2.getAffineTransform(src_point, dst_point)
    border_value = np.median(np.reshape(img, (img.shape[0] * img.shape[1], -1)), axis=0)
    warped_img = cv2.warpAffine(img, transform, dsize=(x, y), borderValue=border_value)
    return (warped_img)

In [3]:
foldername='data/balanced_train'
image_classes = {}
for image_name in os.listdir(foldername):
    class_name = image_name[0:-4].split('_')[0]
    if class_name in image_classes:
        image_classes[class_name].append(image_name)
    else:
        image_classes[class_name] = [image_name]



In [4]:
n=30
for key in image_classes.keys():    
    number_of_samples=len(image_classes.get(key))
    if number_of_samples<n:
        number_of_augment=n-number_of_samples
        original_samples = image_classes.get(key)
        for i in range(number_of_augment):
            selected_sample=original_samples[random.randint(0,number_of_samples-1)]
            augmented_sample=augment_input(cv2.imread(foldername+'/'+selected_sample,0))
            cv2.imwrite(foldername+'/'+selected_sample[:-4]+'a'+str(i)+'.png',augmented_sample)